In [1]:
import numpy as np

In [2]:
def redpr(str):
    print('\x1b[31m' + str + '\x1b[0m')
    
def greenpr(str):
    print('\x1b[32m' + str + '\x1b[0m')
    
def colorpr2(strA, str1, col1, strB, str2, col2):
    print(strA + '\x1b[' + col1 + 'm' + str1.rjust(6) + '\x1b[0m    ', strB.ljust(16) + '\x1b[' + col2 + 'm' + str2.rjust(6) + '\x1b[0m')

In [3]:
week = 14
lastweek = week - 1
wk = str(week)
lwk = str(lastweek)

with open('schedule_2018.txt') as f:
    #passer = f.readlines()
    schedule = list(f)

with open('Week' + lwk + '/SRS_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    SRS = list(f)
    
with open('Week' + lwk + '/YdsOFF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    YdsOFF = list(f)
    
with open('Week' + lwk + '/YdsDEF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    YdsDEF = list(f)
    
with open('Week' + lwk + '/ConvOFF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    ConvOFF = list(f)

with open('Week' + lwk + '/ConvDEF_w' + lwk + '.txt') as f:
    #passer = f.readlines()
    ConvDEF = list(f)

In [4]:
YdsOFF

['<PRE>                                                   \n',
 '                                               Pena\n',
 'Rk                     Tm      G    PF    Yds   Yds\n',
 '1    Tampa Bay Buccaneers     12   318   5312   731\n',
 '2        Los Angeles Rams     12   419   5279   648\n',
 '3      Kansas City Chiefs     12   444   5246   908\n',
 '4     Pittsburgh Steelers     12   346   4965   823\n',
 '5    Los Angeles Chargers     12   340   4793   759\n',
 '6      New Orleans Saints     12   419   4759   623\n',
 '7    New England Patriots     12   331   4741   529\n',
 '8       Green Bay Packers     12   281   4567   728\n',
 '9      Indianapolis Colts     12   325   4535   714\n',
 '10        Atlanta Falcons     12   296   4531   622\n',
 '11      Carolina Panthers     12   304   4522   606\n',
 '12         Houston Texans     12   302   4488   633\n',
 '13       Baltimore Ravens     12   297   4484   653\n',
 '14         Denver Broncos     12   276   4388   682\n',
 '15    S

In [25]:
team_list = []
data_dict = {}

for i in range(len(SRS)):
    
    # SRS:
    team = SRS[i][:22].lstrip().rstrip()
    team_list.append(team)
    
    val = float(SRS[i].rstrip()[-7:])
    
    data_dict[team] = {}
    data_dict[team]['SRS'] = val
    
data_dict

ind1 = YdsOFF[2].index('G')
ind2 = YdsOFF[2].index('PF')
ind3 = YdsOFF[2].index('Yds')
ind4 = YdsOFF[2].index('Yds', ind3 + 1)
league_PF = float(YdsOFF[36][ind2 - 3:ind2 + 2])
league_yds = float(YdsOFF[36][ind3 - 3:ind3 + 3])
league_pen = float(YdsOFF[36][ind4 - 2:ind4 + 3])
PPY = league_PF/(league_yds)
print(league_PF, league_yds, league_pen, PPY)

for i in range(3, len(SRS) + 3):    
    team = YdsOFF[i][2:25].lstrip()
    G = float(YdsOFF[i].rstrip()[ind1 - 1:ind1 + 1])
    PF1 = float(YdsOFF[i].rstrip()[ind2 - 1:ind2 + 2])
    yds = float(YdsOFF[i].rstrip()[ind3 - 1:ind3 + 3])
    pen = float(YdsOFF[i].rstrip()[ind4 :ind4 + 3])
    yds_tot = yds #- pen
    PF2 = yds_tot*PPY
    PPG1_OFF = PF1/G
    PPG2_OFF = PF2/G
    #print(team, G, PF1, yds, pen, yds_tot, PPG1_OFF, PPG2_OFF)
    data_dict[team]['PPG_OFF'] = [PPG1_OFF, PPG2_OFF]
    #print()
    
    team = YdsDEF[i][2:25].lstrip()
    G = float(YdsDEF[i].rstrip()[ind1 - 1:ind1 + 1])
    PA1 = float(YdsDEF[i].rstrip()[ind2 - 1:ind2 + 2])
    yds = float(YdsDEF[i].rstrip()[ind3 - 1:ind3 + 3])
    pen = float(YdsDEF[i].rstrip()[ind4 :ind4 + 3])
    yds_tot = yds #+ pen
    PA2 = yds_tot*PPY
    PPG1_DEF = PA1/G
    PPG2_DEF = PA2/G
    #print(team, G, PA1, yds, pen, yds_tot, PPG1_DEF, PPG2_DEF)
    data_dict[team]['PPG_DEF'] = [PPG1_DEF, PPG2_DEF]
    #print()
    
    team = ConvOFF[i][2:25].lstrip()
    DwnPct = float(ConvOFF[i].rstrip()[26:30])
    RZPct = float(ConvOFF[i].rstrip()[32:36])
    data_dict[team]['Pct_OFF'] = [DwnPct, RZPct]
    #print(DwnPct, RZPct)
    DwnPct = float(ConvDEF[i].rstrip()[26:30])
    RZPct = float(ConvDEF[i].rstrip()[32:36])
    data_dict[team]['Pct_DEF'] = [DwnPct, RZPct]
    #print(DwnPct, RZPct)
    #print()
      
team_list = sorted(team_list)

9186.0 137987.0 21893.0 0.06657148861849305


In [26]:
SRS_ar = []
OFF_ar = []
DEF_ar = []
pow_ar = []

for team in team_list:
    SRS_val = data_dict[team]['SRS']
    rnk_1 = data_dict[team]['PPG_OFF'][0]
    rnk_2 = data_dict[team]['PPG_OFF'][1]
    OFF_val = np.mean([rnk_1, rnk_2])
    rnk_3 = data_dict[team]['PPG_DEF'][0]
    rnk_4 = data_dict[team]['PPG_DEF'][1]
    DEF_val = np.mean([rnk_3, rnk_4])
    
    SRS_ar.append([SRS_val, team]) 
    OFF_ar.append([OFF_val, team, rnk_1, rnk_2])
    DEF_ar.append([DEF_val, team, rnk_3, rnk_4])
    pow_ar.append([np.mean([SRS_val, OFF_val - DEF_val]), team, SRS_val, OFF_val - DEF_val])

    
SRS_rankings = sorted(SRS_ar, reverse=True)
power_OFF = sorted(OFF_ar, reverse=True)
power_DEF = sorted(DEF_ar, reverse=False)
power_rankings = sorted(pow_ar, reverse=True)


print('Power rankings:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), power_rankings[i][1].rjust(22), '{0:+5.1f} {1:+5.1f} {2:+5.1f}'.format(power_rankings[i][0], power_rankings[i][2], power_rankings[i][3]))
    
print()
print('SRS:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), SRS_rankings[i][1].rjust(22), '{0:+5.1f}'.format(SRS_rankings[i][0] ))
    
print()
print('Offense:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), power_OFF[i][1].rjust(22), '{0:4.1f}'.format(power_OFF[i][0]), '   {}'.format(power_OFF[i][2:]))
    team = power_OFF[i][1].lstrip().rstrip()
    data_dict[team]['OFFENSE'] = [i+1, power_OFF[i][0]]
    
print()
print('Defense:')
print()
for i in range(32):
    print('{:2d}'.format(i + 1), power_DEF[i][1].rjust(22), '{0:4.1f}'.format(power_DEF[i][0]), '   {}'.format(power_DEF[i][2:]))
    team = power_DEF[i][1].lstrip().rstrip()
    data_dict[team]['DEFENSE'] = [i+1, power_DEF[i][0]]

Power rankings:

 1     New Orleans Saints +10.1 +12.5  +7.6
 2       Los Angeles Rams  +9.2 +10.9  +7.5
 3     Kansas City Chiefs  +7.8 +10.1  +5.5
 4       Baltimore Ravens  +6.6  +6.6  +6.5
 5   Los Angeles Chargers  +6.3  +6.6  +5.9
 6    Pittsburgh Steelers  +6.1  +6.8  +5.5
 7          Chicago Bears  +5.3  +5.2  +5.4
 8   New England Patriots  +4.3  +4.8  +3.8
 9         Houston Texans  +4.0  +4.1  +3.9
10       Seattle Seahawks  +4.0  +5.9  +2.0
11         Dallas Cowboys  +1.8  +2.2  +1.5
12         Denver Broncos  +1.8  +3.6  +0.0
13     Indianapolis Colts  +1.7  +0.5  +2.9
14      Minnesota Vikings  +0.5  -0.3  +1.4
15      Carolina Panthers  +0.2  -0.4  +0.7
16      Green Bay Packers  -0.4  -1.6  +0.7
17    Philadelphia Eagles  -1.0  -1.4  -0.6
18   Tampa Bay Buccaneers  -1.4  -2.9  +0.0
19       Tennessee Titans  -1.8  -1.5  -2.0
20        Atlanta Falcons  -1.8  -1.5  -2.1
21   Jacksonville Jaguars  -2.1  -3.2  -1.0
22    Washington Redskins  -2.2  -2.0  -2.5
23       Clevel

In [55]:
for i in range(len(schedule)):
    if ( schedule[i][0]==wk and schedule[i][1]==' ' ) or schedule[i][0:2]==wk:
        
        team_away = schedule[i][3:29].lstrip().rstrip()
        team_home = schedule[i][28:].lstrip().rstrip()
        
        spread1 = 2.5 + (- data_dict[team_away]['SRS'] + data_dict[team_home]['SRS']) / 2.
        
        #rnk_1 = data_dict[team_away]['interceptDEF'][1]
        #col_1 = '30'
        #if rnk_1 <= 8:
        #    col_1 = '32'
        #elif rnk_1 >= 25:
        #    col_1 = '31'
        #str_1 = str(rnk_1).rjust(3)
            
        OFF1_away = data_dict[team_away]['PPG_OFF'][0]        
        OFF2_away = data_dict[team_away]['PPG_OFF'][1]
        OFF1_home = data_dict[team_home]['PPG_OFF'][0]
        OFF2_home = data_dict[team_home]['PPG_OFF'][1]
        
        DEF1_away = data_dict[team_away]['PPG_DEF'][0]
        DEF2_away = data_dict[team_away]['PPG_DEF'][1]
        DEF1_home = data_dict[team_home]['PPG_DEF'][0]
        DEF2_home = data_dict[team_home]['PPG_DEF'][1]
        
        Pts_away = np.mean([OFF1_away, OFF2_away, DEF1_home, DEF2_home])
        Pts_home = np.mean([OFF1_home, OFF2_home, DEF1_away, DEF2_away])
        
        spread2 = 2.5 + ( - Pts_away + Pts_home)
        
        spread = (spread1 + spread2) / 2
        total = Pts_away + Pts_home
        score_away = total/2 - spread/2
        score_home = total/2 + spread/2
        
        #print(team_away.rjust(22) + ' {0:+5.1f} ({1:+5.1f}, {2:+5.1f})   at   '.format(spread, spread1, spread2) + team_home )
        print(team_away.rjust(22) + ' {0:+5.1f} ({1:+5.1f})  at  '.format(np.round(2*spread)/2, spread) + team_home, '### total:', np.round(2*total)/2, ',  score {0:2.0f}:{1:2.0f}'.format(np.round(score_away), np.round(score_home)) )
        print()
        print('OFF1: {:5.1f}'.format(OFF1_away), '   OFF2: {:5.1f}'.format(OFF2_away), '            DEF1: {:5.1f}'.format(DEF1_home), '   DEF2: {:5.1f}'.format(DEF2_home))
        print('DEF1: {:5.1f}'.format(DEF1_away), '   DEF2: {:5.1f}'.format(DEF2_away), '            OFF1: {:5.1f}'.format(OFF1_home), '   OFF2: {:5.1f}'.format(OFF2_home))
        print()
        #print('penalties:  {0:3d}'.format(data_dict[team_away]['penalties'][1]).rjust(22))
        #colorpr2('penalties:        ', str_1, col_1, '         ', str_2, col_2)
        #colorpr2('interceptOFF:     ', str_3, col_3, '   interceptDEF:', str_2, col_2)
        #colorpr2('timessacked:      ', str_5, col_5, '   sacks:', str_8, col_8)
        #colorpr2('rushOFF:          ', str_9, col_9, '   rushDEF:', str_12, col_12)
        #colorpr2('3DconvOFF:        ', str_13, col_13, '   3DconvDEF:', str_16, col_16)
        #colorpr2('redzoneOFF:       ', str_17, col_17, '   redzoneDEF:', str_20, col_20)
        #print('----------------------------------------------------------')
        #print('OFFENSE:             {0:3d} ({1:4.1f})'.format(data_dict[team_away]['OFFENSE'][0], data_dict[team_away]['OFFENSE'][1]), \
        #      '  DEFENSE:      {0:3d} ({1:4.1f})'.format(data_dict[team_home]['DEFENSE'][0], data_dict[team_home]['DEFENSE'][1]))
        #print()
        #colorpr2('interceptDEF:     ', str_1, col_1, '   interceptOFF:', str_4, col_4)
        #colorpr2('sacks:            ', str_7, col_7, '   timessacked:', str_6, col_6)
        #colorpr2('rushDEF:          ', str_11, col_11, '   rushOFF:', str_10, col_10)
        #colorpr2('3DconvDEF:        ', str_15, col_15, '   3DconvOFF:', str_14, col_14)
        #colorpr2('redzoneDEF:       ', str_19, col_19, '   redzoneOFF:', str_18, col_18)
        #print('----------------------------------------------------------')
        #print('DEFENSE:             {0:3d} ({1:4.1f})'.format(data_dict[team_away]['DEFENSE'][0], data_dict[team_away]['DEFENSE'][1]), \
        #      '  OFFENSE:      {0:3d} ({1:4.1f})'.format(data_dict[team_home]['OFFENSE'][0], data_dict[team_home]['OFFENSE'][1]))
        #print()
        print()

  Jacksonville Jaguars  +2.5 ( +2.7)  at  Tennessee Titans ### total: 40.5 ,  score 19:22

OFF1:  16.9    OFF2:  22.3             DEF1:  20.4    DEF2:  22.7
DEF1:  20.2    DEF2:  21.0             OFF1:  18.4    OFF2:  20.6


      Baltimore Ravens  +3.0 ( +3.1)  at  Kansas City Chiefs ### total: 52.0 ,  score 24:27

OFF1:  24.8    OFF2:  24.9             DEF1:  27.2    DEF2:  27.8
DEF1:  17.8    DEF2:  18.8             OFF1:  37.0    OFF2:  29.1


       New York Giants  +3.0 ( +3.0)  at  Washington Redskins ### total: 46.0 ,  score 21:24

OFF1:  22.2    OFF2:  23.3             DEF1:  21.4    DEF2:  24.5
DEF1:  26.2    DEF2:  25.1             OFF1:  19.4    OFF2:  21.5


    New Orleans Saints  -3.5 ( -3.3)  at  Tampa Bay Buccaneers ### total: 55.0 ,  score 29:26

OFF1:  34.9    OFF2:  26.4             DEF1:  29.6    DEF2:  26.3
DEF1:  22.4    DEF2:  23.6             OFF1:  26.5    OFF2:  29.5


      Los Angeles Rams  +0.5 ( +0.6)  at  Chicago Bears ### total: 51.5 ,  score 26:26

OFF

In [40]:
np.round?